In [1]:
import torch
import functools

# --- THE FIX: Force Disable PyTorch 2.6 Security Check ---
# This wrapper intercepts the torch.load call and removes the 'weights_only' restriction
# effectively reverting PyTorch to version 2.5 behavior for loading files.
_original_load = torch.load

@functools.wraps(_original_load)
def unsafe_load_wrapper(*args, **kwargs):
    # Remove 'weights_only' if it was passed by the library (Pyannote/WhisperX)
    if 'weights_only' in kwargs:
        del kwargs['weights_only']
    # Call the original load function with security disabled
    return _original_load(*args, **kwargs, weights_only=False)

# Apply the patch
torch.load = unsafe_load_wrapper
# ---------------------------------------------------------

import whisperx
from whisperx.diarize import DiarizationPipeline
import warnings
warnings.filterwarnings("ignore")

print("PyTorch Security Check Disabled. Ready to load models.")

PyTorch Security Check Disabled. Ready to load models.


C:\Users\holmes\.local\share\mamba\envs\agri_transcribe\lib\site-packages\pyannote\audio\core\io.py:212: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  torchaudio.list_audio_backends()


In [2]:
import os
# This environment variable tells PyTorch to allow 'un-safe' globals globally
os.environ["TORCH_FORCE_WEIGHTS_ONLY_LOAD"] = "0"

import gc
import torch
import warnings

# Optional: verify it worked
print(f"PyTorch version: {torch.__version__}")
warnings.filterwarnings("ignore")

# Allowlist for Pyannote/OmegaConf in PyTorch 2.6
from omegaconf.listconfig import ListConfig
from omegaconf.dictconfig import DictConfig
from omegaconf.base import ContainerMetadata
torch.serialization.add_safe_globals([ListConfig, DictConfig, ContainerMetadata])

import whisperx
from whisperx.diarize import DiarizationPipeline

PyTorch version: 2.8.0+cpu


In [ ]:
device = "cuda"
audio_file = r"D:\legendary_interview\251124_102702_TrLR.flac"
batch_size = 16 
compute_type = "float16"
model_dir = "D:/ai_models/whisper_large_v2"

# 1. Load for Danish
model = whisperx.load_model(
    "large-v2", 
    device, 
    compute_type=compute_type, 
    download_root=model_dir, 
    language="da"
)

audio = whisperx.load_audio(audio_file)

# 2. Transcribe with Agricultural Context
# This helps the model choose niche words like 'omdrift' or 'gødning'
agri_prompt = "Landbrug, omdrift, gødning, efterafgrøder, biomasse, landmand, høst."
result = model.transcribe(audio, batch_size=batch_size, initial_prompt=agri_prompt)

# 3. Align (Required for accurate Danish word-level timestamps)
model_a, metadata = whisperx.load_align_model(language_code="da", device=device)
result = whisperx.align(result["segments"], model_a, metadata, audio, device)

# 4. Diarize (Identifying speakers)
diarize_model = DiarizationPipeline(use_auth_token="", device=device)
diarize_segments = diarize_model(audio)
result = whisperx.assign_word_speakers(diarize_segments, result)

# 5. Print Preview
for seg in result["segments"]:
    speaker = seg.get('speaker', 'Unknown')
    print(f"[{seg['start']:>7.2f} - {seg['end']:>7.2f}] {speaker}: {seg['text']}")

2025-12-18 13:58:09 - whisperx.vads.pyannote - INFO - Performing voice activity detection using Pyannote...


Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.6.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint C:\Users\holmes\.local\share\mamba\envs\agri_transcribe\lib\site-packages\whisperx\assets\pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.4.0. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.8.0+cpu. Bad things might happen unless you revert torch to 1.x.


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
import torch
print(f"Version: {torch.__version__}")        # Should say something like '2.5.1+cu124'
print(f"CUDA Available: {torch.cuda.is_available()}") # MUST say 'True'